# Preprocesamiento y preparación del modelo

1. Carga del dataset limpio `spotify_clean_modeling.csv`.  
2. Separación de variables predictoras (X) y objetivo (`y = is_hit`).  
3. Evaluación de los modelos:  
   a. RandomForestClassifier  
   b. GradientBoostingClassifier  
   c. XGBoost  
   d. LightGBM  
   e. LogisticRegression  
   f. KNeighborsClassifier  
4. Escalado o normalización de variables numéricas.  
5. División del conjunto en entrenamiento y prueba (`train_test_split`).  
6. Guardado de los datos procesados (`X_train`, `X_test`, `y_train`, `y_test`).  


In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import os
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import seaborn as sns

# Ruta al archivo fuente inicial 
DATA_PATH = "../data/processed/spotify_clean_modeling.csv"

# Verificar existencia
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"No se encontró el archivo en {DATA_PATH}")

# Carga el archivo CSV
df = pd.read_csv(DATA_PATH)
print(f"Dataset se ha cargado correctamente en un arreglo: {df.shape}")

display(df.columns.T)
display(df.head())


Dataset se ha cargado correctamente en un arreglo: (232724, 13)


Index(['genre', 'popularity', 'acousticness', 'danceability', 'duration_ms',
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'tempo', 'valence', 'is_hit'],
      dtype='object')

,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,is_hit
0,Movie,0,0.611,0.389,99373,0.910,0.000,0.3460,-1.828,0.0525,166.969,0.814,0
1,Movie,1,0.246,0.590,137373,0.737,0.000,0.1510,-5.559,0.0868,174.003,0.816,0
2,Movie,3,0.952,0.663,170267,0.131,0.000,0.1030,-13.879,0.0362,99.488,0.368,0
3,Movie,0,0.703,0.240,152427,0.326,0.000,0.0985,-12.178,0.0395,171.758,0.227,0
4,Movie,4,0.950,0.331,82625,0.225,0.123,0.2020,-21.150,0.0456,140.576,0.390,0


## Creacion Nuevas Caracteristicas y Normalizacion de columnas

In [75]:
# Normalización de Duración

df["duration_min"] = df["duration_ms"] / 60000

df.drop(columns=["duration_ms"],errors='ignore', inplace=True)

# 1. Ritmo percibido real
# Mide cuántos "beats por minuto real" tiene la canción relativo a su duración.
df["beat_density"] = df["tempo"] / df["duration_min"]

# 2. Energía emocional
# Captura qué tan intensa y emocionalmente positiva es la canción a la vez.
df["energy_valence"] = df["energy"] * df["valence"]

# 3. Intensidad bailable
# Representa cuánta energía tiene la canción mientras sigue siendo bailable.
df["dance_energy"] = df["danceability"] * df["energy"]

# 4. Combinaciones adicionales 
# speech_valence → mide cuán "feliz" es una canción hablada o con estilo rap.
df["speech_valence"] = df["speechiness"] * df["valence"]

# acoustic_energy → mide cuánta energía tiene una canción acústica o con instrumentos reales.
df["acoustic_energy"] = df["acousticness"] * df["energy"]

# inst_energy → mide la intensidad de canciones instrumentales, especialmente EDM o techno.
df["inst_energy"] = df["instrumentalness"] * df["energy"]

# dance_valence → mide qué tan bailable y emocionalmente positiva es una canción.
df["dance_valence"] = df["danceability"] * df["valence"]

new_features = [
    "duration_min", "beat_density", "energy_valence", "dance_energy",
        "speech_valence", "acoustic_energy", "inst_energy", "dance_valence"
]

print("Nuevas características creadas:")
for feature in new_features:
    print(f"- {feature}")

display(df[new_features].head())


Nuevas características creadas:
- duration_min
- beat_density
- energy_valence
- dance_energy
- speech_valence
- acoustic_energy
- inst_energy
- dance_valence


,duration_min,beat_density,energy_valence,dance_energy,speech_valence,acoustic_energy,inst_energy,dance_valence
0,1.656217,100.813501,0.740740,0.353990,0.042735,0.556010,0.000000,0.316646
1,2.289550,75.998777,0.601392,0.434830,0.070829,0.181302,0.000000,0.481440
2,2.837783,35.058350,0.048208,0.086853,0.013322,0.124712,0.000000,0.243984
3,2.540450,67.609282,0.074002,0.078240,0.008967,0.229178,0.000000,0.054480
4,1.377083,102.082421,0.087750,0.074475,0.017784,0.213750,0.027675,0.129090


### Actualizacion del modelo.

In [76]:
## Update del Data Set para el Modelo
os.makedirs("../data/processed", exist_ok=True)
df.to_csv("../data/processed/spotify_clean_modeling.csv", index=False)
print("💾 Archivo actualizado en: ../data/processed/spotify_clean_modeling.csv")

💾 Archivo actualizado en: ../data/processed/spotify_clean_modeling.csv


### Columnas finales para training ML

In [77]:
display(df.columns.T)

Index(['genre', 'popularity', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'is_hit', 'duration_min', 'beat_density', 'energy_valence',
       'dance_energy', 'speech_valence', 'acoustic_energy', 'inst_energy',
       'dance_valence'],
      dtype='object')

In [ ]:
print(f"✅ Canciones clasificadas como HIT: {df['is_hit'].sum()} de {len(df)} ({df['is_hit'].mean()*100:.2f}%)")
# Correlación directa con popularidad o is_hit
corr = df.corr(numeric_only=True)
corr["is_hit"].sort_values(ascending=False)

Se encuentra un desbalance de los datos solo 9.6% representan hits, lo que nos hace notar que solo tener un buen accuracy (Predicciones Correctas) no
es suficiente, para el modelo.  

## Separación de variables predictoras (X) y objetivo (y)


In [ ]:

X = df.drop(columns=["is_hit","popularity"])
y = df["is_hit"]


## Creación de DataFrames para Codificar Variabls Categóricas y Entrenamiento de Modelos 


In [ ]:
# Para modelos de árboles → LabelEncoder
X_tree = X.copy()
le = LabelEncoder()
X_tree["genre"] = le.fit_transform(X_tree["genre"])

# Para modelos lineales / distancia → OneHotEncoder
preprocessor_ohe = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), ["genre"])
], remainder="passthrough")

In [ ]:
display(X_tree.dtypes)
X_tree.describe()

## Division de datos

### División en entrenamiento y prueba


In [ ]:
X_train_tree, X_test_tree, y_train, y_test = train_test_split(X_tree, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_tree.describe().T


In [ ]:
X_train_tree["genre"].unique()[:10]


## Definición de Modelos Dinamico

### 1. Logistic Regression
- Modelo lineal.  
- Sirve como baseline.  
- Rápido, interpretable y muestra qué variables empujan a la probabilidad de ser hit.

### 2. Random Forest
- Ensamble de muchos árboles de decisión.  
- Robusto, maneja no-linealidades y detecta interacciones entre features automáticamente.

### 3. Gradient Boosting (GBM clásico de sklearn)
- Construye árboles de manera secuencial, corrigiendo errores del anterior.  
- Mejor rendimiento que RandomForest pero más lento.

### 4. XGBoost
- Implementación optimizada y más poderosa de boosting.  
- Alta precisión, muy usado en competencias de Kaggle.  
- Excelente con datasets tabulares.

### 5. LightGBM
- Boosting muy rápido desarrollado por Microsoft.  
- Funciona excelente con grandes volúmenes (como tu dataset de 230k filas).  
- Suele superar a XGBoost en velocidad con rendimiento similar o mejor.


In [ ]:

# CONFIGURACIÓN GENERAL


# Calcular peso de clase positiva (para XGBoost)
pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

# Colección para guardar resultados de todos los experimentos
resultados_globales = []


# DEFINIR BATCHES PARA LOS MODELOS CON LOS HIPERPARÁMETROS

batch_1 = {
    "RandomForest": {"n_estimators": 300, "max_depth": None, "min_samples_leaf": 2},
    "GradientBoosting": {"n_estimators": 400, "learning_rate": 0.05, "max_depth": 5},
    "XGBoost": {"n_estimators": 600, "learning_rate": 0.05, "max_depth": 6},
    "LightGBM": {"n_estimators": 600, "num_leaves": 64, "learning_rate": 0.03},
    "LogisticRegression": {"max_iter": 1000, "solver": "liblinear"},
    "KNeighbors": {"n_neighbors": 10, "weights": "distance"}
}

batch_2 = {
    "RandomForest": {"n_estimators": 800, "max_depth": 10, "min_samples_leaf": 1},
    "GradientBoosting": {"n_estimators": 800, "learning_rate": 0.02, "max_depth": 6},
    "XGBoost": {"n_estimators": 1000, "learning_rate": 0.03, "max_depth": 8},
    "LightGBM": {"n_estimators": 1000, "num_leaves": 128, "learning_rate": 0.02},
    "LogisticRegression": {"max_iter": 2000, "solver": "liblinear"},
    "KNeighbors": {"n_neighbors": 20, "weights": "uniform"}
}

# Puedes agregar batch_3, batch_4, etc.
batches = {"Batch_1": batch_1, "Batch_2": batch_2}


# FUNCIÓN PARA EJECUTAR UN BATCH
def entrenar_batch(nombre_batch, config_batch):
    resultados = []

    # Modelos tipo árbol
    tree_models = {
        "RandomForest": RandomForestClassifier(n_jobs=-1, random_state=42, class_weight="balanced", **config_batch["RandomForest"]),
        "GradientBoosting": GradientBoostingClassifier(random_state=42, **config_batch["GradientBoosting"]),
        "XGBoost": XGBClassifier(
            n_jobs=-1,
            eval_metric="logloss",
            random_state=42,
            scale_pos_weight=pos_weight,
            **config_batch["XGBoost"]
        ),
        "LightGBM": LGBMClassifier(
            n_jobs=-1,
            random_state=42,
            class_weight="balanced",
            **config_batch["LightGBM"]
        )
    }

    for nombre, modelo in tree_models.items():
        modelo.fit(X_train_tree, y_train)
        y_pred = modelo.predict(X_test_tree)
        resultados.append({
            "Batch": nombre_batch,
            "Modelo": nombre,
            "Accuracy": accuracy_score(y_test, y_pred),
            "F1": f1_score(y_test, y_pred),
            "ROC_AUC": roc_auc_score(y_test, y_pred)
        })

    # Modelos lineales / distancia
    linear_models = {
        "LogisticRegression": LogisticRegression(class_weight="balanced", **config_batch["LogisticRegression"]),
        "KNeighbors": KNeighborsClassifier(n_jobs=-1, **config_batch["KNeighbors"])
    }

    for nombre, modelo in linear_models.items():
        clf = Pipeline(steps=[
            ("preprocess", preprocessor_ohe),
            ("model", modelo)
        ])
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        resultados.append({
            "Batch": nombre_batch,
            "Modelo": nombre,
            "Accuracy": accuracy_score(y_test, y_pred),
            "F1": f1_score(y_test, y_pred),
            "ROC_AUC": roc_auc_score(y_test, y_pred)
        })

    return resultados


# EJECUTAR TODOS LOS BATCHES


for nombre_batch, config in batches.items():
    resultados_globales.extend(entrenar_batch(nombre_batch, config))


# RESULTADOS COMBINADOS


df_resultados = pd.DataFrame(resultados_globales).sort_values(by=["Batch", "F1"], ascending=[True, False])
display(df_resultados)


## Tabla de resultados ordenada por Modelo y Accuracy para cada batch

In [ ]:
df_ordenado = (
    df_resultados
        .sort_values(by=["Modelo", "Accuracy"], ascending=[True, False])
        .reset_index(drop=True)
)

display(df_ordenado)


# Interpretacion de resultados.

## Resumen de los modelos

## 1. GradientBoosting (Batch_2)
**Accuracy:** 0.916  
**F1:** **0.394**  
**ROC-AUC:** 0.632  

**Interpretación:**  
- Accuracy alto, pero engañoso por el desbalance.  
- F1 muy bajo → **no detecta hits correctamente**.  
- AUC ≈ 0.63 → pobre capacidad para separar HIT y NO HIT.  

👉 **Conclusión:** GradientBoosting **no es adecuado** para este problema.

## 2. KNeighbors (Batch_2)
**Accuracy:** 0.909  
**F1:** **0.355**  
**ROC-AUC:** 0.618  

**Interpretación:**  
- Accuracy aceptable, pero no refleja buen rendimiento real.  
- F1 muy bajo → **fracasa detectando hits**.  
- AUC ≈ 0.62 → apenas mejor que un modelo aleatorio.  

👉 **Conclusión:** KNN **es muy débil** para este dataset.

## 3. LightGBM (Batch_2)
**Accuracy:** 0.872  
**F1:** **0.569**  
**ROC-AUC:** **0.869**  

**Interpretación:**  
- Accuracy moderado porque arriesga más (bueno para HIT).  
- F1 alto → **mejor balance entre precisión y recall**.  
- AUC ≈ 0.87 → muy buena capacidad de separabilidad.  

👉 **Conclusión:** LightGBM es **el mejor modelo del batch 2**.

## 4. Logistic Regression (Batch_2)
**Accuracy:** 0.756  
**F1:** **0.415**  
**ROC-AUC:** 0.805  

**Interpretación:**  
- Accuracy bajo, pero normal para un modelo simple.  
- F1 razonable para un baseline.  
- AUC ≈ 0.80 → separación aceptable.  

👉 **Conclusión:** Modelo decente como referencia inicial.

## 5. RandomForest (Batch_2)
**Accuracy:** 0.926  
**F1:** **0.449**  
**ROC-AUC:** 0.676  

**Interpretación:**  
- Accuracy muy alto, pero engañoso por la clase dominante.  
- F1 mediocre → **predice casi siempre NO HIT**.  
- AUC ≈ 0.67 → pobre capacidad para distinguir clases.  

👉 **Conclusión:** RandomForest **no es buena opción** para este problema.

## 6. XGBoost (Batch_2)
**Accuracy:** 0.872  
**F1:** **0.567**  
**ROC-AUC:** **0.866**  

**Interpretación:**  
- Accuracy similar a LightGBM.  
- F1 muy alto → **excelente detección de hits**.  
- AUC ≈ 0.86 → fuerte separabilidad HIT/NO HIT.  

👉 **Conclusión:** XGBoost es **el segundo mejor modelo**, casi empatado con LightGBM.

# 🏆 Conclusión General del Batch 2

| Modelo | Resultado |
|--|--|
| **🥇 LightGBM** | Mejor F1 y mejor AUC |
| **🥈 XGBoost** | Excelente rendimiento, muy cercano a LightGBM |
| Logistic Regression | Baseline aceptable |
| RandomForest | No apto (sesgo hacia clase NO HIT) |
| GradientBoosting | Muy bajo desempeño |
| KNeighbors | Inadecuado para este tipo de datos |

**LightGBM y XGBoost son los modelos recomendados para avanzar al entrenamiento final.**

In [ ]:
df_batch1 = df_resultados[df_resultados["Batch"] == "Batch_1"]
df_batch1.plot(
    x="Modelo",
    y=["Accuracy", "F1", "ROC_AUC"],
    kind="bar",
    figsize=(10,5)
)
plt.title("Comparación de modelos – Predicción de 'Hit' (Batch 1)")
plt.ylabel("Puntaje")
plt.ylim(0,1)
plt.grid(True)
plt.show()

In [ ]:
df_batch2 = df_resultados[df_resultados["Batch"] == "Batch_2"]
df_batch2.plot(
    x="Modelo",
    y=["Accuracy", "F1", "ROC_AUC"],
    kind="bar",
    figsize=(10,5)
)
plt.title("Comparación de modelos – Predicción de 'Hit' (Batch 2)")
plt.ylabel("Puntaje")
plt.ylim(0,1)
plt.grid(True)
plt.show()

## Comparacion de F1-Score Bacth1 

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=df_resultados, x="Modelo", y="F1", hue="Batch")
plt.title("Comparación de F1-score por modelo y batch")
plt.ylabel("F1-score")
plt.ylim(0,1)
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=df_resultados, x="Modelo", y="ROC_AUC", hue="Batch")
plt.title("Comparación de ROC_AUC por modelo y batch")
plt.ylabel("ROC_AUC")
plt.ylim(0,1)
plt.grid(True)
plt.show()

## Seleccion de Modelo

1. Mejor F1-score del Batch 2

El F1 es la métrica más importante porque el dataset está fuertemente desbalanceado (~4–5% hits).

Batch 2 — LightGBM:

F1 = 0.5690 (el más alto de todos los modelos)

Esto significa que LightGBM:

Detecta muchos más hits que otros modelos

Maneja mejor el desbalance

Tiene la mejor combinación de precision + recall para la clase HIT

Comparaciones:

Modelo	F1 (Batch_2)
LightGBM	0.569
XGBoost	0.567
RandomForest	0.449
LogisticRegression	0.415
GradientBoosting	0.394
KNN	0.355

🟢 LightGBM gana el F1-score del Batch 2.

2. AUC muy alto (segundo mejor pero casi empatado)

Batch 2:

LightGBM AUC = 0.8690

Esto indica:

Excelente capacidad para separar HIT vs NO HIT

Umbral de decisión más estable

Curva ROC muy sólida

Comparación:

Modelo	AUC (Batch_2)
LightGBM	0.869
XGBoost	0.866
LogisticRegression	0.805
RandomForest	0.676
GradientBoosting	0.632
KNN	0.618

🟢 LightGBM tiene el mejor AUC del batch (empatado virtualmente con XGBoost).